In [ ]:
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List
import os
import re
import matplotlib

# 1. 解决中文字体显示问题
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial']
    plt.rcParams['axes.unicode_minus'] = False
except:
    pass

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

def natural_sort_key(s: str):
    """
    自然排序键函数，将字符串中的数字部分转换为整数进行比较
    例如: 'pca_10' 会排在 'pca_9' 后面
    """
    def convert(text):
        return int(text) if text.isdigit() else text.lower()
    
    def alphanum_key(key):
        return [convert(c) for c in re.split('([0-9]+)', key)]
    
    return alphanum_key(s)

def sort_model_names(model_names: List[str]) -> List[str]:
    """
    对模型名称进行自然排序
    排序规则:
    1. 首先按类型分组 (Flux_VAE, SD_1_5_VAE)
    2. 在每个类型内按数字自然排序
    """
    # 分离不同类型的模型
    flux_models = [m for m in model_names if m.startswith('Flux_VAE')]
    sd_models = [m for m in model_names if m.startswith('SD_1_5_VAE')]
    other_models = [m for m in model_names if m not in flux_models + sd_models]
    
    # 对每种类型内的模型进行自然排序
    flux_models_sorted = sorted(flux_models, key=natural_sort_key)
    sd_models_sorted = sorted(sd_models, key=natural_sort_key)
    other_models_sorted = sorted(other_models, key=natural_sort_key)
    
    # 组合结果
    return flux_models_sorted + sd_models_sorted + other_models_sorted

def extract_pca_number(model_name: str) -> int:
    """
    从模型名称中提取PCA数字
    例如: 'Flux_VAE_pca_10' -> 10, 'Flux_VAE_original' -> 0
    """
    match = re.search(r'pca_(\d+)', model_name)
    if match:
        return int(match.group(1))
    elif 'original' in model_name:
        return 0
    else:
        return -1

def load_json_files(pattern: str = "vae_reconstruction_metrics_*.json") -> Dict[str, Dict]:
    """
    加载所有匹配模式的JSON文件
    """
    files = glob.glob(pattern)
    data_dict = {}
    
    print(f"Found {len(files)} JSON files:")
    for file in sorted(files):
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                filename = os.path.basename(file)
                model_name = filename.replace('vae_reconstruction_metrics_', '').replace('.json', '')
                data_dict[model_name] = data
                print(f"  ✓ {filename} -> {model_name}")
        except Exception as e:
            print(f"  ✗ Error loading {file}: {e}")
    
    return data_dict

def create_comparison_dataframe(data_dict: Dict[str, Dict]) -> pd.DataFrame:
    """
    将JSON数据转换为对比DataFrame，按自然顺序排序
    """
    # 获取模型名称并按自然顺序排序
    model_names = list(data_dict.keys())
    sorted_model_names = sort_model_names(model_names)
    
    print("\n📊 Model sorting:")
    print(f"Original order: {model_names[:5]}...")
    print(f"Sorted order: {sorted_model_names}")
    
    records = []
    
    for model_name in sorted_model_names:
        metrics = data_dict[model_name]
        record = {'Model': model_name}
        
        # 提取PCA编号（用于后续排序）
        pca_num = extract_pca_number(model_name)
        record['pca_number'] = pca_num
        
        for key, value in metrics.items():
            record[key] = value
        records.append(record)
    
    df = pd.DataFrame(records)
    
    # 确保按照正确的顺序
    df['sort_key'] = df['Model'].apply(natural_sort_key)
    df = df.sort_values('sort_key').drop('sort_key', axis=1)
    
    return df

def create_model_groups(df: pd.DataFrame) -> Dict[str, List[str]]:
    """
    根据模型名称创建分组
    """
    groups = {}
    
    for model_name in df['Model'].tolist():
        if 'Flux_VAE' in model_name:
            if 'original' in model_name:
                group_name = 'Flux_VAE (Original)'
            elif 'pca' in model_name:
                pca_num = extract_pca_number(model_name)
                group_name = f'Flux_VAE (PCA {pca_num})'
            else:
                group_name = 'Flux_VAE'
        elif 'SD_1_5_VAE' in model_name:
            if 'original' in model_name:
                group_name = 'SD 1.5 VAE (Original)'
            else:
                group_name = 'SD 1.5 VAE'
        else:
            group_name = 'Other'
        
        if group_name not in groups:
            groups[group_name] = []
        groups[group_name].append(model_name)
    
    return groups

def plot_metric_comparison(df: pd.DataFrame, metrics: List[str], 
                          metric_names: List[str] = None,
                          figsize=(16, 10)):
    """
    绘制多个指标的对比图，按正确顺序显示
    """
    if metric_names is None:
        metric_names = metrics
    
    n_metrics = len(metrics)
    n_cols = 2
    n_rows = (n_metrics + 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()
    
    # 获取按自然顺序排序的模型列表
    models = df['Model'].tolist()
    
    for idx, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
        ax = axes[idx]
        
        std_metric = f"{metric.split('_')[0]}_std" if '_mean' in metric else f"{metric}_std"
        
        values = df[metric].tolist()
        
        # 创建颜色映射，按模型类型分组
        colors = []
        for model in models:
            if 'Flux_VAE_original' in model:
                colors.append('#1f77b4')  # 蓝色 - 原始模型
            elif 'Flux_VAE_pca' in model:
                pca_num = extract_pca_number(model)
                # PCA模型使用渐变色
                if pca_num <= 5:
                    colors.append('#ff7f0e')  # 橙色 - 低PCA
                elif pca_num <= 10:
                    colors.append('#2ca02c')  # 绿色 - 中PCA
                else:
                    colors.append('#d62728')  # 红色 - 高PCA
            elif 'SD_1_5_VAE' in model:
                colors.append('#9467bd')  # 紫色 - SD模型
            else:
                colors.append('#8c564b')  # 棕色 - 其他
        
        x_pos = np.arange(len(models))
        width = 0.6
        
        if std_metric in df.columns:
            errors = df[std_metric].tolist()
            bars = ax.bar(x_pos, values, width=width, yerr=errors, 
                         capsize=4, alpha=0.8, edgecolor='black', linewidth=1)
        else:
            bars = ax.bar(x_pos, values, width=width, alpha=0.8, edgecolor='black', linewidth=1)
        
        # 设置颜色
        for i, bar in enumerate(bars):
            bar.set_color(colors[i])
        
        ax.set_title(f'{metric_name}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Model', fontsize=10)
        ax.set_ylabel('Value', fontsize=10)
        ax.set_xticks(x_pos)
        
        # 旋转x轴标签，避免重叠
        ax.set_xticklabels(models, rotation=45, ha='right', fontsize=9)
        
        # 添加数值标签
        for i, (v, bar) in enumerate(zip(values, bars)):
            height = bar.get_height()
            y_offset = max(values) * 0.02
            ax.text(bar.get_x() + bar.get_width()/2., height + y_offset,
                   f'{v:.4f}', ha='center', va='bottom', fontsize=8)
        
        # 添加网格
        ax.grid(True, alpha=0.3, linestyle='--', axis='y')
        
        # 根据指标类型添加优化方向说明
        if 'mse' in metric or 'lpips' in metric:
            ax.annotate('Lower is better', xy=(0.98, 0.98), xycoords='axes fraction',
                       ha='right', va='top', fontsize=8, color='red', fontweight='bold')
        else:
            ax.annotate('Higher is better', xy=(0.98, 0.98), xycoords='axes fraction',
                       ha='right', va='top', fontsize=8, color='green', fontweight='bold')
        
        # 调整y轴范围，为标签留出空间
        ax.set_ylim(0, max(values) * 1.15)
    
    # 隐藏多余的子图
    for idx in range(n_metrics, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('VAE Models Reconstruction Metrics Comparison (Natural Order)', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

def plot_trend_analysis(df: pd.DataFrame, metrics: List[str],
                       metric_names: List[str] = None):
    """
    绘制PCA维度变化的趋势图
    """
    if metric_names is None:
        metric_names = metrics
    
    # 分离Flux VAE PCA模型
    pca_models = df[df['Model'].str.contains('Flux_VAE_pca')].copy()
    if len(pca_models) == 0:
        print("No PCA models found for trend analysis")
        return
    
    # 提取PCA编号并排序
    pca_models['pca_num'] = pca_models['Model'].apply(extract_pca_number)
    pca_models = pca_models.sort_values('pca_num')
    
    n_metrics = len(metrics)
    n_cols = 2
    n_rows = (n_metrics + 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 10))
    axes = axes.flatten()
    
    for idx, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        
        x = pca_models['pca_num']
        y = pca_models[metric]
        
        # 绘制折线图
        ax.plot(x, y, 'o-', linewidth=2.5, markersize=8, 
                markerfacecolor='white', markeredgewidth=2, color='#2ca02c')
        
        # 添加数据点标签
        for xi, yi in zip(x, y):
            ax.annotate(f'{yi:.4f}', xy=(xi, yi), xytext=(0, 8),
                       textcoords='offset points', ha='center', fontsize=8)
        
        ax.set_title(f'{metric_name} vs PCA Dimension', fontsize=12, fontweight='bold')
        ax.set_xlabel('PCA Dimension', fontsize=10)
        ax.set_ylabel(metric_name, fontsize=10)
        ax.grid(True, alpha=0.3, linestyle='--')
        
        # 设置x轴刻度
        ax.set_xticks(x)
        ax.set_xticklabels([f'PCA {int(xi)}' for xi in x])
        
        # 添加趋势线
        if len(x) > 1:
            z = np.polyfit(x, y, 1)
            p = np.poly1d(z)
            ax.plot(x, p(x), "--", color='red', alpha=0.5, label=f'Trend: y={z[0]:.4f}x+{z[1]:.4f}')
            ax.legend(fontsize=8)
    
    # 隐藏多余的子图
    for idx in range(len(metrics), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Trend Analysis: Metrics vs PCA Dimension', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

def create_grouped_comparison(df: pd.DataFrame):
    """
    创建分组比较图
    """
    # 创建分组
    groups = create_model_groups(df)
    
    fig, axes = plt.subplots(1, 4, figsize=(20, 6))
    
    metrics = ['mse_mean', 'psnr_mean', 'ssim_mean', 'lpips_mean']
    metric_names = ['MSE', 'PSNR', 'SSIM', 'LPIPS']
    
    for idx, (ax, metric, metric_name) in enumerate(zip(axes, metrics, metric_names)):
        # 为每个分组绘制
        x_labels = []
        group_values = []
        group_colors = []
        
        color_idx = 0
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        for group_name, model_names in groups.items():
            for model_name in model_names:
                if model_name in df['Model'].values:
                    value = df.loc[df['Model'] == model_name, metric].values[0]
                    x_labels.append(f"{group_name}\n{model_name.split('_')[-1]}")
                    group_values.append(value)
                    group_colors.append(colors[color_idx % len(colors)])
            color_idx += 1
        
        bars = ax.bar(range(len(x_labels)), group_values, color=group_colors, 
                     alpha=0.8, edgecolor='black')
        
        ax.set_title(f'{metric_name}', fontsize=12, fontweight='bold')
        ax.set_xticks(range(len(x_labels)))
        ax.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=8)
        
        # 添加数值标签
        for bar, value in zip(bars, group_values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + max(group_values)*0.01,
                   f'{value:.4f}', ha='center', va='bottom', fontsize=7)
    
    plt.suptitle('Grouped Model Comparison', fontsize=16, fontweight='bold', y=1.05)
    plt.tight_layout()
    plt.show()

# 主程序
def main():
    print("=" * 80)
    print("VAE RECONSTRUCTION METRICS VISUALIZATION AND COMPARISON")
    print("=" * 80)
    
    # 1. 加载JSON文件
    print("\n📂 STEP 1: Loading JSON files...")
    data_dict = load_json_files("vae_reconstruction_metrics_*.json")
    
    if not data_dict:
        print("No JSON files found! Please check the file pattern.")
        return
    
    # 2. 创建DataFrame（使用自然排序）
    print("\n📊 STEP 2: Creating comparison dataframe with natural sorting...")
    df = create_comparison_dataframe(data_dict)
    
    print("\n📋 Sorted Model Order:")
    for i, model in enumerate(df['Model'].tolist(), 1):
        print(f"  {i:2d}. {model}")
    
    print("\n📋 Data Overview:")
    print(df.to_string())
    
    # 3. 显示分组信息
    print("\n📊 Model Groups:")
    groups = create_model_groups(df)
    for group_name, model_names in groups.items():
        print(f"  {group_name}: {len(model_names)} models")
    
    # 4. 定义要比较的指标
    mean_metrics = ['mse_mean', 'psnr_mean', 'ssim_mean', 'lpips_mean']
    metric_names = ['MSE (Lower better)', 'PSNR (Higher better)', 
                   'SSIM (Higher better)', 'LPIPS (Lower better)']
    
    # 5. 绘制柱状图（按自然顺序）
    print("\n📈 STEP 3: Creating bar chart comparison (natural order)...")
    plot_metric_comparison(df, mean_metrics, metric_names)
    
    # 6. 绘制趋势分析图
    print("\n📈 STEP 4: Creating trend analysis for PCA models...")
    plot_trend_analysis(df, mean_metrics, metric_names)
    
    # 7. 绘制分组比较图
    print("\n📊 STEP 5: Creating grouped comparison...")
    create_grouped_comparison(df)
    
    # 8. 保存结果
    print("\n💾 STEP 6: Saving results...")
    
    # 保存排序后的数据
    df.to_csv('vae_models_comparison_sorted.csv', index=False, encoding='utf-8-sig')
    
    # 保存分组信息
    with open('model_groups.json', 'w', encoding='utf-8') as f:
        json.dump(groups, f, indent=2, ensure_ascii=False)
    
    print("   ✓ vae_models_comparison_sorted.csv")
    print("   ✓ model_groups.json")
    
    # 9. 显示最佳模型
    print("\n🏆 BEST MODELS FOR EACH METRIC:")
    for metric, metric_name in zip(mean_metrics, ['MSE', 'PSNR', 'SSIM', 'LPIPS']):
        if 'mse' in metric or 'lpips' in metric:
            best_row = df.loc[df[metric].idxmin()]
        else:
            best_row = df.loc[df[metric].idxmax()]
        
        print(f"  {metric_name}: {best_row['Model']} = {best_row[metric]:.6f}")
    
    print("\n" + "=" * 80)
    print("✅ ANALYSIS COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    
    return df

# 执行主程序
if __name__ == "__main__":
    result_df = main()
    
    # 显示最终DataFrame
    print("\n📄 FINAL SORTED COMPARISON TABLE:")
    display(result_df.style
           .background_gradient(subset=['mse_mean', 'lpips_mean'], cmap='RdYlGn_r')
           .background_gradient(subset=['psnr_mean', 'ssim_mean'], cmap='RdYlGn')
           .format({'mse_mean': '{:.6f}', 'psnr_mean': '{:.6f}', 
                   'ssim_mean': '{:.6f}', 'lpips_mean': '{:.6f}'}))